### Segment semantic synthetic dataset

In [ ]:
import cv2
from pathlib import Path

# -------- CONFIG ----------
images_dir = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/GRAPE-FOCUS_Mix_2025-09-09_vSemantic/segmented"
labels_dir = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/GRAPE-FOCUS_Mix_2025-09-09_vSemantic/labels-segmented/"
convert_gimphsv_to_cv2hsv = True
min_area = 50   # filter out tiny blobs
epsilon_ratio = 0.01    # 1% of arc length → simplifies polygons

# HSV ranges (lower, upper, class_name)
color_ranges = [
    ((238, 56, 80), (240, 58, 82), "grape"), # class 0
    ((7, 84, 85), (8, 85, 86), "black_rot"),
    ((25, 74, 82), (27, 76, 84), "esca"),
    ((252, 55, 81), (253, 56, 82), "downy_mildew"),
    ((340, 63, 83), (341, 64, 84), "powdery_mildew"),
    ((55, 46, 78), (56, 47, 79), "leaf_blight"), # class 5
]

# Map class names to YOLO IDs
class_to_id = {c[2]: i for i, c in enumerate(color_ranges)}


def convert_hsv_gimp_to_cv2(H_gimp, S_gimp, V_gimp):
    H_cv2 = int(H_gimp / 2)
    S_cv2 = int(S_gimp * 255 / 100)
    V_cv2 = int(V_gimp * 255 / 100)
    return (H_cv2, S_cv2, V_cv2)


# -------- PROCESS IMAGES ----------
Path(labels_dir).mkdir(parents=True, exist_ok=True)

for img_file in sorted(Path(images_dir).glob("*.*")):
    img = cv2.imread(str(img_file))
    if img is None:
        continue

    h, w, _ = img.shape
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    label_lines = []

    for lower, upper, name in color_ranges:
        lw, up = lower, upper
        if convert_gimphsv_to_cv2hsv:
            lw = convert_hsv_gimp_to_cv2(*lower)
            up = convert_hsv_gimp_to_cv2(*upper)

        mask = cv2.inRange(hsv, lw, up)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < min_area:
                continue

            # Simplify polygon
            epsilon = epsilon_ratio * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)

            if len(approx) < 3:
                continue

            # Normalize segmentation points
            norm_points = []
            for [x, y] in approx.reshape(-1, 2):
                px = x / w
                py = y / h
                norm_points.extend([px, py])

            class_id = class_to_id[name]
            line = f"{class_id} " + " ".join(f"{p:.6f}" for p in norm_points)
            label_lines.append(line)

    # Save YOLO-Seg labels
    if label_lines:
        label_path = Path(labels_dir) / (img_file.stem + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(label_lines))

print(f"✅ Done! YOLO-Seg annotations saved in {labels_dir}")


### Draw segmentation

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# -------- CONFIG ----------
images_dir = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/GRAPE-FOCUS_Mix_2025-09-09_vSemantic/images"
labels_dir = "/home/lucas-fontoura/Documents/yolo-drone/data/datasets/GRAPE-FOCUS_Mix_2025-09-09_vSemantic/labels-segmented"
output_dir = "vis_labels"   # where to save visualization images
class_names = ["grape", "black_rot", "esca", "downy_mildew", "powdery_mildew", "leaf_blight"]   # keep in sync with your dataset classes
num_samples = 20    # number of images to process

Path(output_dir).mkdir(parents=True, exist_ok=True)

# -------- FUNCTION ----------
def visualize_yolo_seg(img_path, label_path, save=True):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠ Could not read {img_path}")
        return

    h, w, _ = img.shape

    if not Path(label_path).exists():
        print(f"⚠ No label for {img_path.name}")
        return

    with open(label_path, "r") as f:
        lines = f.read().strip().splitlines()

    for line in lines:
        parts = line.split()
        cls_id = int(parts[0])
        coords = list(map(float, parts[1:]))

        # Convert normalized coords back to pixel values
        points = []
        for i in range(0, len(coords), 2):
            x = int(coords[i] * w)
            y = int(coords[i + 1] * h)
            points.append([x, y])

        points = cv2.convexHull(np.array(points))  # ensure closed polygon
        cv2.polylines(img, [points], isClosed=True, color=(0, 255, 0), thickness=2)

        # Label with class name
        if class_names:
            cx, cy = points[0][0]
            cv2.putText(img, class_names[cls_id], (cx, cy - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    if save:
        out_path = Path(output_dir) / img_path.name
        cv2.imwrite(str(out_path), img)
        print(f"✅ Saved visualization to {out_path}")
    else:
        # Show inline with matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.axis("off")
        plt.show()


# -------- RUN ON ALL IMAGES ----------
import numpy as np

for img_file in sorted(Path(images_dir).glob("*.*"))[:num_samples]:
    label_file = Path(labels_dir) / (img_file.stem + ".txt")
    visualize_yolo_seg(img_file, label_file, save=True)
